In [1]:
import lzma
import os
import pickle

import numpy as np
from PIL import Image

### File I/O

In [2]:
def unpickle_data(file):
    """
    Unpickle file using bz2 and pickle.
    :param file: file path

    :return: data from file
    """
    if not os.path.exists(file):
        return None
    with lzma.open(file, 'rb') as fo:
        return pickle.load(fo)
    return

In [3]:
data_path = 'data/dataset_compressed_1.pkl'
dataset = unpickle_data(data_path)
print(type(dataset['labels']))
print(type(dataset['labels'][0]))
print(type(dataset['data']))
print(type(dataset['data'][0]))
print((dataset['data'][0]).shape)

<class 'list'>
<class 'int'>
<class 'list'>
<class 'numpy.ndarray'>
(24, 24, 3)


### Dataset

In [4]:
from src.reader import load_dataset, DigitDataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [5]:

base_path = 'data/dataset_compressed'
start_idx = 1
end_idx = 6
X_train, Y_train, X_test, Y_test = load_dataset(base_path, start_idx, end_idx)


In [6]:

# Define transformations (if needed)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # TODO: adjust
])

# Create PyTorch datasets
train_dataset = DigitDataset(X_train, Y_train, transform=transform)
test_dataset = DigitDataset(X_test, Y_test, transform=transform)

# Data loaders for use in training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# pick a random image from the dataset, and display it with its label usibg matplotlib
import matplotlib.pyplot as plt
import random

idx = random.randint(0, len(train_dataset))
img, label = train_dataset[idx]
img = img.permute(1, 2, 0)
plt.imshow(img)
plt.title(f'Label: {label}')
plt.show()


### Model

In [8]:
import torch
import torch.nn as nn

from src.model import NeuralNet

### Trainer

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
def train(model, train_loader, epochs=10):
    # decay the learning rate by a factor of 0.1 every 30 epochs
    scheduler = torch.optim.lr_scheduler.StepLR(model.optimizer, step_size=35, gamma=0.1)
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).type(torch.long)
            loss = model.step(images, labels)
            total_loss += loss
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

    # Update the learning rate scheduler
    scheduler.step()


In [ ]:

# Initialize the neural network
lrate = 0.01  # Learning rate.

# Loss function
loss_fn = nn.CrossEntropyLoss()                 # Accuracy: 0.85733
# loss_fn = nn.MultiMarginLoss()                 # Accuracy: 0.84667

# Example of using the train function
model = NeuralNet(lrate=0.01, loss_fn=loss_fn, in_size=3*24*24, out_size=10)
model = model.to(device)

train(model, train_loader, epochs=100)


In [15]:
# Evaluate the model on the test set and Confusion Matrix
def test(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    # confusion_matrix as int tensor
    confusion_matrix = torch.zeros(10, 10, dtype=torch.int)
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.type(torch.int64) == labels).sum().item()
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    print(f"Accuracy: {(correct / total) * 100:.5f}%")
    print("Confusion Matrix:")
    print(confusion_matrix)

test(model, test_loader)

Accuracy: 99.60000%
Confusion Matrix:
tensor([[ 939,    0,    0,    1,    0,    0,    1,    1,    1,    0],
        [   0, 1231,    1,    1,    3,    2,    1,    1,    1,    0],
        [   0,    0, 1071,    1,    0,    1,    0,    1,    0,    0],
        [   0,    1,    0, 1028,    1,    0,    0,    2,    0,    1],
        [   0,    1,    0,    0, 1225,    0,    0,    0,    0,    2],
        [   0,    1,    2,    2,    1, 1116,    1,    1,    0,    0],
        [   1,    1,    0,    1,    0,    0, 1229,    0,    1,    0],
        [   0,    0,    1,    0,    1,    1,    0, 1129,    1,    0],
        [   0,    0,    0,    1,    0,    0,    0,    0, 1070,    0],
        [   0,    1,    1,    0,    0,    1,    0,    0,    0, 1167]],
       dtype=torch.int32)


In [13]:
# Save the model
torch.save(model.state_dict(), 'data/model.pth')
